# Loading data into the vectorDB

In [2]:
!pip3 install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 6.8 MB/s eta 0:00:00


In [ ]:
from dotenv import dotenv_values
SECRETS=dotenv_values(".env")

In [ ]:
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=secrets['PINECONE_API_KEY'],
    envrionment=secrets['PINECONE_ENV']
    
)

index_name="langchain-demo"
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "my-vector-db"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536, 
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    ) 

